Note: il est recommandé de travailler avec RStudio. A l'ouverture de ce document R markdown (.Rmd), si les accents apparaissent mal, dans RStudio sélectionner File, Reopen with encoding puis utf8. Une fois le document réalisé, vous pouvez utiliser le bouton knit pour générer une sortie au format html.

La numérotation des exercices poursuit celle du TP n°1 qui comporte trois exercices, elle commence donc à 4.

On rappelle une syntaxe de base pour opérer un Krigeage Simple à l'aide de la librairie DiceKriging:
`monModele <- km(formula = ~1, design = X, response = Y, covtype = "gauss", coef.trend = 0,  coef.cov = theta, coef.var = sigma2)`
`prediction <- predict(object = monModele, newdata = xnew , type="SK" , checkNames=FALSE, se.compute=TRUE)`.

# Exercice 4. Optimisation des covariances en dimension 1

On considère un phénomène physique, pour les besoins du TP, nous le modéliserons par une fonction
$$ f(x) = \sin(2x)+\log(2+x) $$
on observe cette fonction à des abscisses tirées aléatoirement entre -1 et 1.

Créer un échantillon d'apprentissage de $n=100$ évaluations de la fonction, et un échantillon de test de $q=50$ évaluations de la fonction. 

In [1]:

# tapez votre code ou vos commentaires ici


On suppose que le paramètre de portée est $\theta \in [0.5, 5]$.

Pour calculer les prédicteurs de Krigeage, nous utiliserons DiceKriging comme dans le TP1.

## 4a. Erreur de validation croisée

Pour un paramètre $\theta$ donné et une famille de covariance "matern3_2" faire une fonction qui:

* au moyen d'un Krigeage Simple opère une prévision de la fonction au $q$ points de l'échantillon test.

* renvoie l'erreur quadratique moyenne entre ces prédictions et les vraies valeurs de la fonction.

Cette erreur dépend-elle de la variance du champ (paramètre `coef.var=` dans `km`)?

In [2]:
# tapez votre code ou vos commentaires ici


## 4b. Optimisation de la portée sur échantillon test

Tracer l'évolution de l'erreur obtenue en fonction du paramètre de portée, et trouver le minimum approximatif et l'erreur qui en résulte. Répéter ce traitement pour plusieurs familles de covariance. Peut-on identifier une portée optimale? essayer de faire varier `set.seed(...)` en début de votre programme, pour avoir une idée de la robustesse de cette portée optimale.

In [3]:
# tapez votre code ou vos commentaires ici


## 4c. Autres techniques: MLE et LOO

Dans DiceKriging, lorsque les paramètres de portée (*lengthscales* ou encore *range*) ne sont pas précisés, une optimisation est effectuée pour les estimer. Il est alors possible de les obtenir au moyen de  `coef(krigingModel, "range")`, où `krigingModel`est un output de la fonction `km`. La méthode utilisée est configurable grâce à l'argument `estim.method="MLE"` ou `estim.method="LOO"`.	

Utiliser DiceKriging pour estimer le paramètre de portée optimal, par l'une ou l'autre des méthodes.

In [4]:
# tapez votre code ou vos commentaires ici


Une estimation de la variance est possible de la même façon via  `coef(krigingModel, "sd2")`. Donner la variance et l'écart-type estimés par DiceKriging

In [5]:
# tapez votre code ou vos commentaires ici

# Exercice 5

Dans certains cas, il est possible d'observer des répétitions du champ aléatoire considéré. Nous allons voir dans cet exercice comment optimiser les paramètres de portée pour s'approcher d'une corrélation empirique observée.

Dans le répertoire du TP, un fichier *SriLankaTP2.csv* donne des précipitations mensuelles observées dans différentes ville au Sri Lanka.

De nombreuses analyses de série chronologiques seraient possibles (saisonnalité, valeurs aberrantes, etc.), pour les besoins du TP, nous travaillerons directement sur les données brutes et non sur des résidus. 

## 5a. Données et visualisations
Importer tout d'abord le fichier (attention au séparateur et à la première ligne). Afficher les correlations au moyen de la fonction `cor`. Vous pourrez aussi utiliser la fonction `pairs` .

In [6]:
# tapez votre code ou vos commentaires ici

## 5b. Matrice des distances

Après une recherche rapide sur Internet, on trouve les latitudes et longitudes approximatives suivantes:
Colombo: 6.922, 79.863
Pamban: 9.279, 79.225
Puttalam: 8.0348, 79.837
Thiruvananthapuram: 8.660, 76.946
Trincomalee: 8.591, 81.213

Créer une matrice `X`à deux colonnes contenant pour chaque ville ces coordonnées géographiques. Au moyen de `as.matrix(dist(X))`, créer ensuite une matrice contenant pour chaque paire de ville, une distance entre ces villes (peu importe l'unité), aisni que le coefficient de corrélation empirique entre les précipitations des deux villes en questions.

Remarque sur la distance: on pourrait utiliser une distance géodésique sur la sphère, mais il faudrait alors adapter le noyau de covariance pour obtenir une fonction semi-définie positive. Autour de la ville de Colombo, 1 degré de latitude représente 111km, contre 110km pour 1 degré de longitude; nous négligerons également cette différence dans le TP. Au final, on ne considèrera donc qu'un distance euclidienne simple entre des coordonnées (lat, long), et un seul paramètre de portée.


In [7]:

# tapez votre code ou vos commentaires ici

On suppose désormais que l'on utilise un noyau Gaussien isotrope de sorte que la correlation entre deux villes à distance $d$ est:
$$ r(d) =\exp\left(-d^2/(2\theta^2)\right)$$

## 5c. Corrélations théoriques

Créer une fonction R renvoyant la matrice de corrélation théorique pour un $\theta$ donné.

In [8]:
# tapez votre code ou vos commentaires ici


## 5d. Optimisation de la portée sur correlations

Optimiser le paramètre de portée pour minimiser un écart de votre choix entre les corrélations théoriques (avec ce noyau et cette portée), et les corrélations empiriques. Nommez votre portée optimale *optimalTheta*

In [9]:
# tapez votre code ou vos commentaires ici


## 5e Prédiction

Avec cette famille de covariance la portée optimale, prédire au moyen d'un krigeage ordinaire les précipitations dans la ville de *Dambulla* en février 2013, en sachant que les précipitations pour ce mois dans les 5 villes respectives étaient

Colombo: 164, 
Pamban: 62, 
Puttalam: 93, 
Thiruvananthapuram: 43, 
Trincomalee: 355.

Note: il vous faudra ici appliquer la portée $\theta$ à la latitude comme à la longitude, de sorte que `coef.cov = c(optimalTheta, optimalTheta)`.

In [10]:

# tapez votre code ou vos commentaires ici


On pourrait opérer une prédiction en de nombreux points pour obtenir une carte de l'intensité des précipitations à cette période. Il serait ainsi possible de tracer en couleur l'intensité des précipitations prédites sur toute la région. Bien sûr, le modèle est ici très frustre, il ne tient pas compte de la géographie (mers, montagnes), de l'évolution temporelle, et de bien d'autres phénomènes physiques...

Par ailleurs, si on observait les précipations au nouveau point de prédiction *Dambulla*, il vaudrait mieux essayer d'expliquer ce niveau en fonction des cinq autres niveaux, pour exploiter toutes les observations, en chaque date. Ici on suppose qu'on ne dispose pas de cette information (nombre de stations limité p.ex., comme dans le contexte minier, où le nombre de forages est limité).

# Exercice 6

Vous trouverez dans le répertoire de Campus un fichier intitulé "defi_observations.csv", il comporte des observations pour 5 variables X1, ..., X5, et une variable à prédire Y.

Vous trouverez également dans le répertoire du cours un fichier intitulé "defi_apredire.csv" comportant des valeurs pour les 5 variables X1, ..., X5, et où il faudra prédire Y.

Remettre vos prédictions dans un fichier csv comportant les 5 variables X1, ..., X5, et la colonne Y prédite. Votre fichier s'intitulera "DefiGroupeN" où vous remplacerez bien sûr le suffixe N par le numéro de votre groupe. Le programme à l'origine de vos prédictions sera initulé "programmeDefiGroupeN". Quel que soit le langage utilisé, vérifier que votre fichier est lisible sous R avec la commande `read.csv("DefiGroupeN.csv", header = TRUE)`.
Par exemple, pour exporter votre fichier csv sous R, vous utiliserez la commande `write.csv(votreDataFrame,"DefiGroupeN.csv", row.names = FALSE)`. 

Le programme du meilleur groupe sera posté sur Campus.
La contrainte: la prédiction doit se faire au moyen du Krigeage (eh oui, c'est un TP de Krigeage).

ATTENTION! il vous faudra faire attention à bien utiliser une graine pour votre générateur aéatoire, p.ex `set.seed(12345)` de façon à ce que vos résultats soit reproductibles. D'une exécution à l'autre, votre programme doit proposer LA MEME prédiction!